In [1]:
pip install transformers datasets torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from datasets import load_dataset

# Load the IMDB dataset
imdb_data = load_dataset("imdb")

# Split into training and testing sets
train_texts = imdb_data["train"]["text"]
train_labels = imdb_data["train"]["label"]
test_texts = imdb_data["test"]["text"]
test_labels = imdb_data["test"]["label"]

print(f"Number of training samples: {len(train_texts)}")
print(f"Number of testing samples: {len(test_texts)}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Number of training samples: 25000
Number of testing samples: 25000


In [3]:
from transformers import BertTokenizer

# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Tokenize training and test data
train_encodings = tokenizer(
    train_texts, truncation=True, padding=True, max_length=128, return_tensors="pt"
)
test_encodings = tokenizer(
    test_texts, truncation=True, padding=True, max_length=128, return_tensors="pt"
)


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [4]:
from torch.utils.data import DataLoader, TensorDataset
import torch

# Convert labels to tensors
train_labels = torch.tensor(train_labels)
test_labels = torch.tensor(test_labels)

# Create TensorDataset
train_dataset = TensorDataset(
    train_encodings["input_ids"], train_encodings["attention_mask"], train_labels
)
test_dataset = TensorDataset(
    test_encodings["input_ids"], test_encodings["attention_mask"], test_labels
)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)


In [5]:
from transformers import BertForSequenceClassification, AdamW

# Load the BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
model = model.to("cuda")  # Use GPU if available

# Optimizer and loss function
optimizer = AdamW(model.parameters(), lr=5e-5)

# Training loop
from torch.nn import CrossEntropyLoss
from tqdm import tqdm

epochs = 3  # Number of epochs
loss_fn = CrossEntropyLoss()

for epoch in range(epochs):
    model.train()
    total_loss = 0
    print(f"Epoch {epoch + 1}/{epochs}")

    for batch in tqdm(train_loader):
        # Move data to GPU
        input_ids, attention_mask, labels = (
            batch[0].to("cuda"),
            batch[1].to("cuda"),
            batch[2].to("cuda"),
        )

        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Training loss: {total_loss / len(train_loader)}")


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/3


100%|██████████| 1563/1563 [09:19<00:00,  2.79it/s]


Training loss: 0.3419866580528017
Epoch 2/3


100%|██████████| 1563/1563 [09:25<00:00,  2.76it/s]


Training loss: 0.19743364151511805
Epoch 3/3


100%|██████████| 1563/1563 [09:25<00:00,  2.76it/s]

Training loss: 0.10647149789523958


In [6]:
from sklearn.metrics import classification_report

model.eval()
predictions, true_labels = [], []

with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, labels = (
            batch[0].to("cuda"),
            batch[1].to("cuda"),
            batch[2].to("cuda"),
        )

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = torch.argmax(logits, axis=1)

        predictions.extend(preds.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Print classification report
print("\nClassification Report:\n")
print(classification_report(true_labels, predictions, target_names=["Negative", "Positive"]))



Classification Report:

              precision    recall  f1-score   support

    Negative       0.92      0.78      0.85     12500
    Positive       0.81      0.94      0.87     12500

    accuracy                           0.86     25000
   macro avg       0.87      0.86      0.86     25000
weighted avg       0.87      0.86      0.86     25000



In [8]:
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    inputs = {key: val.to("cuda") for key, val in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
        prediction = torch.argmax(probs, dim=-1).item()

    return "Positive" if prediction == 1 else "Negative"

# Test with user input
user_input = input("Enter a movie review: ")
print(f"Sentiment: {predict_sentiment(user_input)}")


Enter a movie review: Unbelievably bad acting," and "Poor direction."
Sentiment: Negative
